In [2]:
!conda install -c pytorch-lts pytorch


^C


In [1]:
import sys

In [2]:
sys.path

['C:\\Users\\User\\Desktop\\מדמח\\מודלימז\\פרויקט חדש',
 'C:\\Users\\User\\Downloads\\pytorch-normalizing-flows-master\\pytorch-normalizing-flows-master\\nflib',
 'C:\\Users\\User\\try',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\plotbrowser-0.1-py3.9.egg',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\Pythonwin']

In [3]:
sys.path.append(r"c:\users\user\anaconda3\envs\pytorch_env\lib\site-packages")


In [4]:
import sys

In [5]:
sys.path

['C:\\Users\\User\\Desktop\\מדמח\\מודלימז\\פרויקט חדש',
 'C:\\Users\\User\\Downloads\\pytorch-normalizing-flows-master\\pytorch-normalizing-flows-master\\nflib',
 'C:\\Users\\User\\try',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\plotbrowser-0.1-py3.9.egg',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\Pythonwin']

In [4]:
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

import torch.nn as nn
import torch.nn.functional as F
import time
from util import findStatistics
from sklearn.preprocessing import LabelEncoder

In [7]:
df=pd.read_csv(r"C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\archive (4)\neo_v2.csv")

In [8]:
df

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,False,16.73,False
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,False,20.00,True
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,False,17.83,False
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,False,22.20,False
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,False,20.09,True
...,...,...,...,...,...,...,...,...,...,...
90831,3763337,(2016 VX1),0.026580,0.059435,52078.886692,1.230039e+07,Earth,False,25.00,False
90832,3837603,(2019 AD3),0.016771,0.037501,46114.605073,5.432121e+07,Earth,False,26.00,False
90833,54017201,(2020 JP3),0.031956,0.071456,7566.807732,2.840077e+07,Earth,False,24.60,False
90834,54115824,(2021 CN5),0.007321,0.016370,69199.154484,6.869206e+07,Earth,False,27.80,False


In [5]:
le = LabelEncoder()
df=pd.read_csv(r"C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\archive (4)\neo_v2.csv")

# groups observations by id, then sums the hazardous column
grouped = df.groupby("id").sum()["hazardous"]

    
# this is a binary variable that measure whether there are more than one hazardous observation for that id
df["prev_hazard"] = df.apply(lambda row: grouped[row["id"]] > 1, axis="columns")


yColumn = ["hazardous"]
xColumns = ["relative_velocity", "absolute_magnitude", "miss_distance", "prev_hazard"]
df = df[xColumns + yColumn]

X=df.apply(le.fit_transform)
X=X.drop(columns=['hazardous'],axis=0)
y=df['hazardous']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [13]:
parameters = {'max_depth':list(range(50)), 'min_samples_leaf':list(range(2,50,1)),'criterion':["gini", "entropy"]}

et=time.time()
model= DecisionTreeClassifier()
clf = GridSearchCV(model, parameters)

clf.fit(X_train, y_train)
ft=time.time()

#print("elapsed time {}".format(ft-et))
print("score: {}".format(clf.score(X_test,y_test)))

score: 0.995082749256908


In [14]:
print("elapsed time {}".format(ft-et))


elapsed time 2726.1002538204193


In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(15,32)
        self.fc2=nn.Linear(32,64)
        self.fc6=nn.Linear(64,128)
        self.fc7=nn.Linear(128,256)
        self.fc8=nn.Linear(256,128)
        self.fc3=nn.Linear(128,32)
        self.fc4=nn.Linear(32,8)
        self.fc5=nn.Linear(8,1)


    def forward(self,x):
        
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = F.relu(self.fc3(x))
        
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x
    
     
    
    
net = Net()
net.load_state_dict(torch.load(r'C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\batches117'))
#best_max_depth,best_min_samples_leaf,best_criterion=params[np.argmax(scores)]
#best_criterion='entropy' if best_criterion==1 else 'gini'

<All keys matched successfully>

In [16]:
scores=[]

et=time.time()
statistics=np.array(findStatistics(X_train,y_train))
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            hyperTensor=np.array([max_depth,min_samples_leaf,criterion])
            combtensor=torch.tensor(np.concatenate((statistics,hyperTensor))).float()
            
            score_predicted=net(combtensor).detach().numpy()
            scores.append(score_predicted)
params=[]
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            params.append((max_depth,min_samples_leaf,criterion)) 
            
scores=np.array(scores).flatten()
params=np.array(params)

tenLargetIdx = (-scores).argsort()[:10]
bests=params[tenLargetIdx]   


res=[]
for max_depth,min_samples_leaf,criterion in bests:
    criterion='entropy' if criterion==1 else 'gini'
    model= DecisionTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion)
    model.fit(X_train,y_train)
    res.append(model.score(X_test,y_test))
    

best=np.argmax(np.array(res))
best_max_depth,best_min_samples_leaf,best_criterion=bests[best]
best_criterion='entropy' if best_criterion==1 else 'gini'
model2= DecisionTreeClassifier(max_depth=best_max_depth,min_samples_leaf=best_min_samples_leaf,criterion=best_criterion)

model2.fit(X_train,y_train)
ft=time.time()
print("elapsed time {}".format(ft-et))

print('score: {}'.format(model2.score(X_test,y_test)))
            


elapsed time 39.059250831604004
score: 0.9951194451579759


In [17]:
best_max_depth

8